<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Evaluate machine learning models</h1>
<h3 align="right">Written by the KSO Team</h3>

# Set up KSO requirements

### Install all the requirements

If you are running this notebook in Google Colab, this cell should take ~2 mins and might restart the session. Please run this cell until you get the "Successful installation!" message.

In [1]:
# Check if notebook is running in colab
import sys, os

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # Clone kso repo and install requirements
    if not os.path.exists("kso"):
        print("Installing all dependencies...")
        !git clone --recurse-submodules -b master https://github.com/ocean-data-factory-sweden/kso.git
        %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso/requirements.txt)

    # Restart colab session if conflicting libraries can't be imported
    try:
        import panoptes_client
    except:
        os.kill(os.getpid(), 9)

    # Enable external widgets and navigate to the kso tutorial folder
    try:
        from google.colab import output

        output.enable_custom_widget_manager()
        os.chdir("kso/tutorials")
    except:
        pass

# Prepare the dev settings if needed
try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
        import kso_utils

        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Avoid issues with widgets not displaying properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet


# Load the clear output function to keep things clean
from IPython.display import clear_output

clear_output()
print("Successful installation... you're good to go!")

Successful installation... you're good to go!


### Import python packages

In [2]:
# Import required modules for tut#6
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
import kso_utils.tutorials_utils as t_utils
import kso_utils.yolo_utils as y_utils
from kso_utils.project import ProjectProcessor, MLProjectProcessor

print("Packages loaded successfully")

ERROR:root:Modules yolov5 and yolov5_tracker are required for ML functionality.


Packages loaded successfully


### Choose your project

In [5]:
project_name = kso_widgets.choose_project()

Dropdown(description='Project:', options=('Template project', 'Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'…

### Initiate project's database

In [6]:
# Find project
project = p_utils.find_project(project_name=project_name.value)
# Initialise pp
pp = ProjectProcessor(project)

INFO:root:Template project loaded succesfully
INFO:root:Retrieving the file from https://drive.google.com/uc?&confirm=s5vl&id=1PZGRoSY_UpyLfMhRphMUMwDXw4yx1_Fn
Downloading...
From: https://drive.google.com/uc?&confirm=s5vl&id=1PZGRoSY_UpyLfMhRphMUMwDXw4yx1_Fn
To: /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/db_csv_info.zip
100%|██████████| 3.23k/3.23k [00:00<00:00, 3.68MB/s]
INFO:root:Updated species table from the temporary database
INFO:root:Updated sites table from the temporary database
INFO:root:Updated movies table from the temporary database


In [32]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

# Evaluate model on custom footage

### Choose the model

In [8]:
os.environ["MLFLOW_TRACKING_URI"]="https://vendor.cloudina.org/"
os.environ["MLFLOW_TRACKING_USERNAME"]="admin@cloudina.org"
os.environ["MLFLOW_TRACKING_PASSWORD"]="WlZg13rPNyqvW6pCjSoaaUmUK166CUoTTfS9c2E1j86AIWpuwRWEIEEsUspsAWfP"

In [118]:

model = mlp.choose_model()

Dropdown(description='Select model: ', options=('mlflow-artifacts:/1/f12f24491c074d42a6769b5ab1cf26b9/artifact…

### Choose folder to download the model

In [34]:
download_dir = kso_widgets.choose_folder(".", "where to download the model")

FileChooser(path='.', filename='', title='HTML(value='Choose location of where to download the model')', show_…

In [35]:
# @title <font size="5">↓ ឵឵឵឵<i>Download model</font> { vertical-output: true }
artifact_dir = mlp.get_model(model.value, download_dir.selected)

In [109]:
# @title <font size="5">↓ ឵឵឵឵<i>Choose your custom footage</font> { vertical-output: true }
source = kso_widgets.choose_footage(
    project=pp.project,
    server_connection=pp.server_connection,
    db_connection=pp.db_connection,
    start_path=pp.project.movie_folder
    if pp.project.movie_folder not in [None, "None"]
    else ".",
    folder_type="custom footage",
)

FileChooser(path='.', filename='', title='HTML(value='Choose location of custom footage')', show_hidden='False…

In [110]:
source_value = t_utils.process_source(source)

In [38]:
# @title <font size="5">↓ ឵឵<i>Choose a confidence threshold for evaluation</font> { vertical-output: true }
conf_thres = t_utils.choose_conf()

FloatSlider(value=0.5, continuous_update=False, description='Confidence threshold:', max=1.0, readout_format='…

In [134]:
# @title <font size="5">↓ ឵឵<i>Run model over selected custom footage</font> { vertical-output: true }
mlp.detect_yolo(
    source=source_value,
    conf_thres=conf_thres.value,
    artifact_dir=artifact_dir,
    save_output=True,
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-ROV 2000 Säcken revet EJ numrerade band_blur.mp4: 512x640 5 deep_water_corals, 1403.8ms
video 1/1 (2/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-ROV 2000 Säcken revet EJ numrerade band_blur.mp4: 512x640 6 deep_water_corals, 1336.8ms
video 1/1 (3/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-RO

In [114]:
# @title <font size="5">↓ ឵឵<i>Choose folder with the evaluation data</font> { vertical-output: true }
eval_dir = kso_widgets.choose_folder(
    save_dir.selected
    if "save_dir" in vars() and save_dir.selected is not None
    else ".",
    "runs output",
)

FileChooser(path='.', filename='', title='HTML(value='Choose location of runs output')', show_hidden='False', …

In [119]:
# @title <font size="5">↓ ឵឵<i>Add the data to Weights and Biases</font> { vertical-output: true }
mlp.save_detections(conf_thres.value, model.value, eval_dir.selected)

INFO:root:Report created at /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/runs/detect/predict7/annotations.csv


In [120]:
# @title <font size="5">↓ ឵឵<i>View model output</font> { vertical-output: true }
viewer = y_utils.choose_files(eval_dir.selected)

Dropdown(description='Select file:', layout=Layout(width='50%'), options=('/Users/jurie.germishuys/Workspace/o…

Output()

In [121]:
# @title <font size="5">↓ ឵឵<i>Investigate training and validation datasets (only image data)</font> { vertical-output: true }
train_dataset, val_dataset = mlp.get_dataset(model.value)

ERROR:root:This is not currently supported for MLflow


In [55]:
# Training set
y_utils.get_data_viewer(os.path.join(train_dataset, "data/images"))

FileNotFoundError: [Errno 2] No such file or directory: 'data/images'

In [ ]:
# Validation set
y_utils.get_data_viewer(os.path.join(val_dataset, "data/images"))

# Track unique individuals (Optional)

In [ ]:
results = mlp.track_yolo(
    source=source_value,
    artifact_dir=artifact_dir,
    conf_thres=conf_thres.value,
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-ROV 2000 Säcken revet EJ numrerade band_blur.mp4: 128x128 5 deep_water_corals, 115.3ms
video 1/1 (2/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-ROV 2000 Säcken revet EJ numrerade band_blur.mp4: 128x128 5 deep_water_corals, 87.1ms
video 1/1 (3/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-ROV 2

In [135]:
results = mlp.track_individuals(name=mlp.project_name,
                                source=source_value,
                                artifact_dir=artifact_dir,
                                eval_dir=eval_dir.selected,
                                conf_thres=conf_thres.value,
                                img_size=(128, 128))



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-ROV 2000 Säcken revet EJ numrerade band_blur.mp4: 128x128 5 deep_water_corals, 89.7ms
video 1/1 (2/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-ROV 2000 Säcken revet EJ numrerade band_blur.mp4: 128x128 5 deep_water_corals, 95.7ms
video 1/1 (3/250) /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/000203 TMBL-ROV 20

MOT results saved to /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/runs/detect/predict7/runs/track/template_project/mot/000203 TMBL-ROV 2000 Säcken revet EJ numrerade band_blur.mp4.txt
this is the latest tracker /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/runs/detect/predict7/runs/track/template_project/mot


INFO:root:Report created at /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/runs/detect/predict7/tracking.csv
INFO:root:------- DETECTION REPORT -------
INFO:root:--------------------------------
INFO:root:species_name
deep_water_coral    13
Name: tracker_id, dtype: int64


                                               filename class_id frame_no  \
0     000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0        1   
1     000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0        1   
2     000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0        1   
3     000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0        1   
4     000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0        1   
...                                                 ...      ...      ...   
6076  000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0      250   
6077  000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0      250   
6078  000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0      250   
6079  000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0      250   
6080  000203 TMBL-ROV 2000 Säcken revet EJ numrerad...        0      250   

     tracker_id  
0             1  
1             2  
2             3  
3  

In [ ]:
# END